# Pitchfork album reviews and song lyrics analysis
# Formatting the data
---

In [1]:
import pandas as pd
import pickle
import json
import os

## Load the Pitchfork reviews

In [2]:
# Pitchfork album reviews
filename_album_reviews = "./data/album_reviews.pickle"
if os.path.exists(filename_album_reviews) and os.path.getsize(filename_album_reviews) > 0:
    artist_reviews = pickle.load(open(filename_album_reviews, 'rb'))

## Convert the reviews from JSON string to Pitchfork Review format

In [9]:
from pitchfork import pitchfork as pf
from bs4 import BeautifulSoup

In [10]:
def convert_json_to_review(review_as_string):
    json_obj = json.loads(review_as_string)
    return pf.Review(json_obj['searched_artist'], json_obj['searched_album'],
                     json_obj['matched_artist'], json_obj['matched_album'],
                     json_obj['query'], json_obj['url'], BeautifulSoup(json_obj['soup'], "lxml"))

In [ ]:
# Convert to Review format
all_reviews = {}
for name, albums in artist_reviews.items():
    print(name)
    album_reviews = []
    for album, review in albums.items():
        album_reviews.append(convert_json_to_review(review))
    all_reviews[name] = album_reviews
    
# List of all artist names with reviews
artist_names_with_reviews = list(all_reviews.keys())
print('\nDone.')

In [15]:
all_reviews.keys()

dict_keys(['Kanye West', 'The Beatles', '2Pac', 'Eminem', 'The Notorious B.I.G.', 'Nas', 'Ice Cube', 'JAY-Z', 'Snoop Dogg', 'Dr. Dre', 'Kendrick Lamar', 'A$AP Rocky', '50 Cent', 'Busta Rhymes', 'J. Cole', 'Method Man', 'KRS-One', 'Ludacris', 'Lil Wayne', 'Common', 'Ghostface Killah', 'T.I.', 'Lauryn Hill', 'Scarface', 'Raekwon', 'The Game', "Ol' Dirty Bastard", 'Mobb Deep', 'Logic', 'Chance The Rapper', 'Cypress Hill', 'Lupe Fiasco', 'RZA', 'GZA', 'Warren G', 'Q-Tip', 'Talib Kweli', 'Missy Elliott', 'Immortal Technique', 'Big Sean', 'Kid Cudi', 'Twista', 'Big Boi', 'Drake', 'Wiz Khalifa', 'Eric B. & Rakim', 'ScHoolboy Q', 'Run-D.M.C.', 'Hopsin', 'Nelly', 'Tyler, The Creator', 'Jadakiss', 'Kurupt', 'Grandmaster Flash & The Furious Five', 'N.W.A', 'Gang Starr', 'E-40', 'Pusha T', 'De La Soul', 'Yelawolf', 'Earl Sweatshirt', 'Fabolous', 'Mac Miller', 'Jeezy', 'Kodak Black', '2 Chainz', 'Chief Keef', 'Gucci Mane', 'Salt-N-Pepa', "Lil' Kim", 'Nicki Minaj', 'M.I.A.', 'Azealia Banks', 'Angel 

## Convert Pitchfork Reviews to JSON format

In [105]:
reviews_json = []
for artist, album_reviews in all_reviews.items():
    print(40 * '-')
    print(artist)
    artist_reviews = []
    for review in album_reviews:
        review_json_obj = {}
        review_method_names = [m for m in dir(r) if (not m.find('_') == 0) and (m != 'soup')]
        for method in review_method_names:
            try:
                review_json_obj[method] = getattr(review, method)();
            except:
                review_json_obj[method] = None
                
        artist_reviews.append(review_json_obj)                
    reviews_json.append({'artist': artist, 'reviews': artist_reviews})

----------------------------------------
Kanye West
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
The Beatles
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
ye

score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
Lil Wayne
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
f

score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
GZA
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
Warren G
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
Q-Tip
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text(

----------------------------------------
E-40
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
Pusha T
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
De La Soul
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
Yelawolf
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
sco

----------------------------------------
Angel Haze
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
DeJ Loaf
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
Jean Grae
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
Ciara
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()


----------------------------------------
Red Hot Chili Peppers
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
Michael Jackson
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
alb

score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
Santana
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
The Who
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
John Lennon
score()
abstract()
best_new_music()
editorial()
f

score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
----------------------------------------
David Bowie
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()
full_text()
cover()
artist()
album()
label()
year()
score()
abstract()
best_new_music()
editorial()

In [110]:
# Save the JSON object
with open('./data/pitchfork_reviews.json', 'w') as outfile:
    json.dump(reviews_json, outfile)